In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import os
from PIL import Image
from pickle import dump, load
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Step 1: Load and preprocess images
def load_images(images_folder, save_file_to=None):
    X_original = []
    X = []
    y = []

    for subdir, dirs, files in os.walk(images_folder):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(subdir, file)
                label = os.path.basename(subdir)

                image = Image.open(img_path).convert('L')  # Convert to grayscale
                image = image.resize((48, 48))  # Resize to 48x48
                X_original.append(np.array(image).flatten())
                X.append(np.array(image).flatten())
                y.append(label)

    if save_file_to:
        with open(save_file_to, "wb") as f:
            dump((X_original, X, y), f, protocol=5)

    return np.array(X_original), np.array(X), np.array(y)

images_folder = "../images"
dataset_file = "dataset_dump.pkl"


X_original, X, y = load_images(images_folder, save_file_to=dataset_file)

In [ ]:
with open("dataset_dump.pkl", "rb") as f:
    X_original,X,y = load(f)
    
X
y

In [ ]:
# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
# Preprocessing data function
def preprocessing_data(X, y, save_file_to=None):
    # Normalize pixel values
    X = X / 255.0  # Normalize to [0, 1]

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=100)  # Adjust the number of components as needed
    X_pca = pca.fit_transform(X_scaled)

    if save_file_to:
        with open(save_file_to, "wb") as f:
            dump((X_pca, y_encoded, label_encoder, scaler), f, protocol=5)

    return X_pca, y_encoded, label_encoder, scaler

# Process the data and save
X_scaled, y_encoded, label_encoder, scaler = preprocessing_data(X, y, save_file_to="preprocessed_data.pkl")

In [ ]:
with open("preprocessed_data.pkl", "rb") as f:
    X_scaled, y_encoded, label_encoder, pca, scaler = load(f)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Custom dataset to handle PCA data
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Reshape the PCA data as an image for visualization purposes
        image = self.X[idx].reshape(10, 10)  # Example reshape (adjust based on PCA output)
        
        # Apply the transformation if defined
        if self.transform:
            image = self.transform(image)

        label = self.y[idx]
        # Ensure the image is of type float32
        return image.type(torch.float32), label  # This line is added

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(10),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


# Create DataLoader
train_dataset = CustomDataset(X_train, y_train, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)  # Increased neurons
        self.fc2 = nn.Linear(512, 256)        # Added extra layer
        self.fc3 = nn.Linear(256, len(np.unique(y_encoded)))  # Output layer
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))  # Apply relu after the second layer
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [ ]:
# Initialize the model, loss function, and optimizer
model = SimpleNN(input_size=100)  # Adjust based on PCA output size
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Add learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images.view(images.size(0), -1))  # Flatten images for FC layer
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    scheduler.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
# Evaluation and prediction
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    y_pred = predicted.numpy()

# Calculate accuracy and print classification report
accuracy = accuracy_score(y_test_tensor.numpy(), y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test_tensor.numpy(), y_pred, target_names=label_encoder.classes_))

In [ ]:
# Visualize predictions
def visualize_predictions(model, X_test, y_test, label_encoder, X_original):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, y_pred = torch.max(outputs.data, 1)

    random_indices = np.random.randint(0, len(y_test), 5)

    for idx in random_indices:
        original_image = X_original[idx].reshape(48, 48)  # Reshape original image for visualization
        plt.imshow(original_image, cmap='gray')
        plt.title(f"True: {label_encoder.inverse_transform([y_test[idx]])[0]}, "
                  f"Predicted: {label_encoder.inverse_transform([y_pred[idx]])[0]}")
        plt.axis('off')
        plt.show()

# Call visualization function
visualize_predictions(model, X_test_tensor, y_test_tensor, label_encoder, X_original)